In [1]:
pip install newspaper3k

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: sgmllib3k in c:\users\sawanta\anaconda3\lib\site-packages (from feedparser>=5.2.1->newspaper3k) (1.0.0)



In [2]:
import pandas as pd
import newspaper
from newspaper import Article
import time
from time import sleep
from datetime import date
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

start_time = time.time()
dte=date.today()
dte1=dte.strftime("%d%m%Y")
date=dte.strftime("%d-%m-%Y")

# output file path
path="C:/Users/sawanta/OneDrive - Dun and Bradstreet/Desktop/Work Files/News/NewsandSentimentsAnalysis_9Sept.xlsx"
wb=pd.ExcelWriter(path)
cols = ['Date', 'URL', 'Category', 'Headline', 'News']
datalists = dict()
for item in cols:
    datalists[item] = []

# News paper link    
url="https://economictimes.indiatimes.com"

try:
    pg=requests.get(url, verify=False)
    soup=BeautifulSoup(pg.content, 'lxml')
    prtify=soup.prettify()
    link=soup.find_all("a")
    for lnk in link:
        href=lnk.get('href')
        if href is None:
            continue
        if "/articleshow/" in href:
            if "/hindi/" not in href:
                if "/news/economy/" in href:
                    news=href
                elif "/markets/" in href:
                    if "/expert-view/" not in href:
                        news=href
                elif "/Small-biz/" in href:
                    news=href
                elif "/industry/" in href:
                    if "entertainment" not in href:
                        if "electronics" not in href:
                            if "/advertising/" not in href:
                                news=href
                elif "/tech/ites/" in href:
                    news=href
                elif "/small-biz/" in href:
                    news=href
                else:
                    continue
                if "https" not in news:
                    news=url+news
                    #print('URL-',news)
                    # Appending data into datalists
                    datalists['URL'].append(news)
                    datalists['Date'].append(date)
                    cat1=news.split(".com")[1]
                    cat2=cat1.split("/")
                    cat_fin=cat2[1]+"/"+cat2[2]#+"/"+cat2[3]print("Category: "+cat_fin)
                    # Appending data into datalists
                    datalists['Category'].append(cat_fin)
                    #print('Category-',cat_fin)
                    arti=Article(news)
                    try:
                        pg2=requests.get(news, verify=False)
                        soup2=BeautifulSoup(pg2.content, 'lxml')
                        prtify2=soup2.prettify()
                        head=prtify2.split('"headline": "')[1]
                        headline=head.split('"description":')[0]
                        headline=headline.rstrip().strip('",')
                        #print('headline')
                        datalists['Headline'].append(headline)
                        arti=prtify2.split('"articleBody":"')[1]
                        txt=arti.split('"image": {')[0]
                        txt=txt.rstrip().strip('",')
                        # Appending data into datalists
                        datalists['News'].append(txt)
                        print('Article')
                    except:
                        pass
                else:
                    print('Without HTTP -',news)
except:
    pass

Article
Article
Article
Article
Article
Article
Article
Article
Article
Without HTTP - https://economictimes.indiatimes.com/industry/energy/oil-gas/russia-to-flood-asia-with-fuel-as-europe-ramps-up-sanctions/articleshow/94324166.cms
Article
Article
Without HTTP - https://economictimes.indiatimes.com/small-biz/entrepreneurship/curiousjr-initiates-a-new-plan-for-coding-in-vernacular-languages/articleshow/94319875.cms
Without HTTP - https://economictimes.indiatimes.com/small-biz/entrepreneurship/curiousjr-initiates-a-new-plan-for-coding-in-vernacular-languages/articleshow/94319875.cms
Without HTTP - https://economictimes.indiatimes.com/small-biz/entrepreneurship/curiousjr-initiates-a-new-plan-for-coding-in-vernacular-languages/articleshow/94319875.cms
Article
Article
Article
Article
Article
Article
Article
Article
Article
Article
Article
Article
Article
Article
Article
Article
Article
Article
Article
Article
Without HTTP - https://economictimes.indiatimes.com/news/economy/finance/fm-nirma

In [3]:

## Final Block of code - keep as it is
data = pd.DataFrame.from_dict(datalists, orient='index')
data = data.transpose()
data.to_excel(wb, sheet_name="Sheet1", index=False)
wb.save()
print("--- %s seconds ---" % (time.time() - start_time))

--- 1037.2071249485016 seconds ---


In [1]:
import pandas as pd
    
df1 = pd.read_excel(r"C:\Users\sawanta\Downloads\NewsAndSentiment_Aug2022_Final (1).xlsx")
df1.shape

(3676, 17)

In [2]:
new_df = pd.DataFrame()
new_df['Date'] = df1['Date']
new_df['Headline'] = df1['Headline']
new_df['URL'] = df1['URL']
new_df['Category'] = df1['Category']
new_df['News'] = df1['News'].astype(str)

In [4]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import en_core_web_sm 
nlp = en_core_web_sm.load()
nlp.add_pipe('spacytextblob')
import string
punctuations = string.punctuation

test_text = new_df['News']
def spacy_tokenizer(test_text):
    mytokens = nlp(test_text)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens

# Basic function to clean the text 
def clean_text(text):     
    return text.strip().lower()

In [3]:
from textblob import TextBlob

def sentimental(text: str) -> str:
    sentiment = None
    if text:
        text = ''.join(text.split()).strip() # removing empty strings
        blob = TextBlob(text)
        if blob.sentiment.polarity > 0:
            sentiment = 'Positive'
        if blob.sentiment.polarity < 0:
            sentiment = 'Negative'
        if blob.sentiment.polarity == 0:
            sentiment = 'Neutral'
    return sentiment

In [4]:
''''''polarity_score = []

for text in test_text:
  doc = nlp(text)
  polarity = (doc._.polarity)
  polarity_score.append(polarity)
''''''

In [5]:
import numpy as np
new_df['Compound'] = polarity_score
conditions = [
    (new_df['Compound'] > 0.05),
    (new_df['Compound'] < -0.05),
    (new_df['Compound'] >= -0.05) & (new_df['Compound'] <= 0.05)]

values = ['Positive', 'Negative', 'Neutral']

new_df['Sentiment'] = np.select(conditions, values)
new_df

,Date,Headline,URL,Category,News,Compound,Sentiment
0,01-08-2022,India's 1st 5G spectrum auction concludes on d...,https://economictimes.indiatimes.com/industry/...,industry/telecom,India's first ever 5G spectrum auctions has en...,-0.019139,Neutral
1,01-08-2022,July GST revenue at ₹1.48 lk cr; up 28% YoY an...,https://economictimes.indiatimes.com/news/econ...,news/economy,The gross goods and service tax (GST) revenue ...,0.154545,Positive
2,01-08-2022,India's manufacturing PMI at 8-month high as e...,https://economictimes.indiatimes.com/news/econ...,news/economy,India's manufacturing sector witnessed an upsu...,0.055661,Positive
3,01-08-2022,Home loan borrowers face higher outgo in goodb...,https://economictimes.indiatimes.com/news/econ...,news/economy,Home loan borrowers are bracing for more incre...,0.099076,Positive
4,01-08-2022,This Big Bull company turned profitable after ...,https://economictimes.indiatimes.com/markets/s...,markets/stocks,"Rakesh Jhunjhunwala-promoted Star Health, at l...",0.147115,Positive
...,...,...,...,...,...,...,...
3671,30-08-2022,'I thought PM Modi to be a crude man but he sh...,https://timesofindia.indiatimes.com/india/i-th...,/india,"""sEzxj hide\n ""_2sdAr contentwrappe",0.000000,Neutral
3672,30-08-2022,Chinese soldiers stop Indian graziers in Demch...,https://timesofindia.indiatimes.com/india/chin...,/india,"""sEzxj hide\n ""_2sdAr contentwrappe",0.000000,Neutral
3673,30-08-2022,PM Modi to attend first in-person Quad summit ...,https://timesofindia.indiatimes.com/india/pm-m...,/india,"""sEzxj hide\n ""_2sdAr contentwrappe",0.000000,Neutral
3674,30-08-2022,50 J&K Congress leaders quit party in support ...,https://timesofindia.indiatimes.com/india/50-j...,/india,"""sEzxj hide\n ""_2sdAr contentwrappe",0.000000,Neutral


In [22]:
df = pd.read_excel(r"C:\Users\sawanta\Downloads\NewsData.xlsx")
test_text = df['News']
def spacy_tokenizer(test_text):
    mytokens = nlp(test_text)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens

# Basic function to clean the text 
def clean_text(text):     
    return test_text.strip().lower()

In [ ]:
from textblob import TextBlob
def sentimental(text: str) -> str:
    sentiment = None
    if text:
        text = ''.join(text.split()).strip() # removing empty strings
        blob = TextBlob(text)
        if blob.sentiment.polarity > 0:
            sentiment = 'Positive'
        if blob.sentiment.polarity < 0:
            sentiment = 'Negative'
        if blob.sentiment.polarity == 0:
            sentiment = 'Neutral'
    return sentiment

In [23]:
polarity_score_test = []

for text in test_text:
  doc = nlp(text)
  polarity = (doc._.polarity)
  polarity_score_test.append(polarity)

In [24]:
df['Compound'] = polarity_score_test

In [27]:
import numpy as np
conditions = [
    (df['Compound'] > 0.05),
    (df['Compound'] < -0.05),
    (df['Compound'] >= -0.05) & (df['Compound'] <= 0.05)]

values = ['Positive', 'Negative', 'Neutral']

df['Sentiment'] = np.select(conditions, values)
df

,Date,URL,Category,Headline,News,Compound,Sentiment
0,22-06-2022,https://economictimes.indiatimes.com/news/econ...,news/economy,An export overhaul could see SEZs shifting fro...,India is proposing to transform its narrow exp...,0.133157,Positive
1,22-06-2022,https://economictimes.indiatimes.com/industry/...,industry/indl-goods,Tata Steel to become an intellectual property ...,Tata Steel will diversify beyond manufacturing...,0.069859,Positive
2,22-06-2022,https://economictimes.indiatimes.com/industry/...,industry/banking,Fintechs behaving like banks may have triggere...,The central bank's latest communication to fin...,0.065837,Positive
3,22-06-2022,https://economictimes.indiatimes.com/markets/s...,markets/stocks,"Multibagger in making? Down 50%, this Rakesh J...","NEW DELHI: Jubilant Ingrevia, whose stock has ...",0.135624,Positive
4,22-06-2022,https://economictimes.indiatimes.com/industry/...,industry/telecom,India set to have 70-80 million 5G phones by t...,India will have 70-80 million 5G smartphones b...,0.076795,Positive
...,...,...,...,...,...,...,...
100,22-06-2022,https://economictimes.indiatimes.com/markets/s...,markets/stocks,Jhunjhunwala’s top 5 stock bets in bear grip! ...,NEW DELHI: Big Bull Rakesh Jhunjhunwala’s top ...,0.067765,Positive
101,22-06-2022,https://economictimes.indiatimes.com/markets/c...,markets/cryptocurrency,"Crypto Price Today: Shiba Inu, Dogecoin zoom u...",New Delhi: Crypto market was trading mixed on ...,0.091856,Positive
102,22-06-2022,https://economictimes.indiatimes.com/markets/s...,markets/stocks,Jhunjhunwala’s top 5 stock bets in bear grip! ...,NEW DELHI: Big Bull Rakesh Jhunjhunwala’s top ...,0.067765,Positive
103,22-06-2022,https://economictimes.indiatimes.com/markets/c...,markets/commodities,Gold rate today: Yellow metal edges higher; si...,NEW DELHI: Gold prices edged higher on Tuesday...,0.127302,Positive


In [33]:
trainData = new_df.drop('News', axis = 1)
testData = df.drop('News', axis = 1)
train_vectors = vectorizer.fit_transform(new_df['News'])
test_vectors = vectorizer.transform(df['News'])

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

In [35]:
## Count Vectorizer Method 
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

ctmTr = cv.fit_transform(trainData)
X_test_dtm = cv.transform(testData)

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model = LogisticRegression()
model.fit(ctmTr, train_vectors)
y_pred_class = model.predict(X_test_dtm)

ValueError: y should be a 1d array, got an array of shape () instead.

In [30]:
# importing libraries
from sklearn import svm
from sklearn.metrics import classification_report


# Initialising SVM classifier with linear kernel
svm_classifier = svm.SVC(kernel='linear')

# training the model with the train data

svm_classifier.fit(train_vectors, trainData['Sentiment'])

 
# testing the model in test data content

predicted_result = svm_classifier.predict(test_vectors)

# results

report = classification_report(testData['Sentiment'], predicted_result, output_dict=True)

print('Model accuracy: ', report['accuracy'])

Model accuracy:  0.7714285714285715


In [7]:
new_df.to_excel(r'C:\Users\sawanta\Downloads\News_Output.xlsx')

# Comparison DF

In [13]:
comparison_df = pd.merge(df1, new_df, on = 'News')
comparison_df = comparison_df[['News', 'News_sentiment', 'News_senti_score','Compound','Sentiment']]
comparison_df

,News,News_sentiment,News_senti_score,Compound,Sentiment
0,India's first ever 5G spectrum auctions has en...,Positive,0.4528,-0.019139,Neutral
1,The gross goods and service tax (GST) revenue ...,Positive,0.8555,0.154545,Positive
2,India's manufacturing sector witnessed an upsu...,Positive,0.9854,0.055661,Positive
3,Home loan borrowers are bracing for more incre...,Positive,0.8934,0.099076,Positive
4,"Rakesh Jhunjhunwala-promoted Star Health, at l...",Positive,0.9988,0.147115,Positive
...,...,...,...,...,...
298123,Prioritize work for ‘18 hours a day for at lea...,Positive,0.8830,0.072526,Positive
298124,"With a market valuation of ₹20.68 Cr, Saianan...",Positive,0.9219,-0.017106,Neutral
298125,Prime Minister Narendra Modi will commission I...,Positive,0.9829,0.162028,Positive
298126,Adani Transmission has entered the coveted lis...,Positive,0.8833,0.218352,Positive


In [16]:
neutral_to_positive = comparison_df[(comparison_df.News_sentiment == "Neutral") & (comparison_df.Sentiment == "Positive")]
ngative_to_neutral = comparison_df[(comparison_df.News_sentiment == "Negative") & (comparison_df.Sentiment == "Neutral")]
negative_to_positive = comparison_df[(comparison_df.News_sentiment == "Negative") & (comparison_df.Sentiment == "Positive")]


662


In [20]:
dflist= [new_df, neutral_to_positive, ngative_to_neutral, negative_to_positive]

Excelwriter = pd.ExcelWriter(r"C:\Users\sawanta\OneDrive - Dun and Bradstreet\Desktop\MSME_Email\Comparison_News_Sentiment.xlsx",engine="xlsxwriter")

for i, df in enumerate (dflist):
    df.to_excel(Excelwriter, sheet_name= str(i),index=False)
#And finally we save the file
Excelwriter.save()

In [11]:
comparison_df.columns

Index(['Date_x', 'URL_x', 'Category_x', 'Headline_x', 'Headline_sentiment',
       'Headline_senti_score', 'Headline_pos_wrds', 'Headline_neu_wrds',
       'Headline_neg_wrds', 'News', 'News_sentiment', 'News_senti_score',
       'News_key_phrases', 'News_pos_wrds', 'News_neu_wrds', 'News_neg_wrds',
       'Words', 'Date_y', 'Headline_y', 'URL_y', 'Category_y', 'Compound',
       'Sentiment'],
      dtype='object')